In [806]:
from pyqubo import Binary, Array, Num
import numpy as np
from dimod import ExactSolver
import neal
from gen import gen_average_degree_directed, gen_planted_path, StandardGraph
from typing import List, Tuple, Callable
from math import log2, floor

In [807]:
def make_tail_graph(graph : StandardGraph, tail_length : int) -> StandardGraph:

	if tail_length == 0:
		tail_graph = StandardGraph(0, [])
		tail_graph.vertices += graph.vertices
		tail_graph.edges = graph.edges.copy()
		return tail_graph

	tail_graph = StandardGraph(0, [])
	tail_graph.vertices += graph.vertices + tail_length
	tail_graph.edges = graph.edges.copy()
	
	for i in range(0,graph.vertices):
		tail_graph.edges.append((i,graph.vertices))
	
	for i in range(graph.vertices , graph.vertices + tail_length - 1):
		tail_graph.edges.append((i, i+1))

	return tail_graph


def make_vertex_edges(graph : StandardGraph) -> List[List[Tuple[int, int, int]]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, v_2) = graph.edges[i]
		
		out[v_1].append((i,v_1,v_2))
		out[v_2].append((i,v_1,v_2))
	
	return out

In [808]:
# graph = StandardGraph(5, [
# 	(0,1),
#     (1,2),
#     (2,3),
#     (3,4),
# ])

graph = gen_planted_path(100, 0.9)

# max length
N = graph.vertices
K = N - 1

P = -N**2

#binaries = range(floor(log2(K)) + 1)

tail_graph = make_tail_graph(graph, K)

vertex_edges = make_vertex_edges(graph)

In [809]:


vertex_vars = Array.create('vertex', tail_graph.vertices, vartype='BINARY')
edge_vars = Array.create('edge', len(tail_graph.edges), vartype='BINARY')
initial_edge_vars = Array.create('init_edge', tail_graph.vertices, vartype='BINARY')
terminal_edge_vars = Array.create('term_edge', tail_graph.vertices, vartype='BINARY') 

# flow_vars = Array.create('flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY')
# flow_vars = Array.create('contra_flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY')


candy_exp = sum(edge_vars[i] for i in range(len(graph.edges)))

initial_node_exp = P * (1 - sum(initial_edge_vars[i] for i in range(graph.vertices)))**2

terminal_node_exp = P * (1 - sum(terminal_edge_vars[i] for i in range(graph.vertices)))**2

all_node_deg_2_exp = 0
for i in range(graph.vertices):
	temp = sum(edge_vars[vertex_edges[i][j][0]] for j in range(len(vertex_edges[i])))
	temp += initial_edge_vars[i] + terminal_edge_vars[i] + edge_vars[graph.vertices + i]
	print(temp)
	all_node_deg_2_exp += P * (2 * vertex_vars[i] - temp)**2






((Binary('init_edge[0]') + Binary('term_edge[0]')) + Binary('edge[905]') + Binary('edge[812]') + Binary('edge[801]') + Binary('edge[484]') + Binary('edge[478]') + Binary('edge[177]') + 0.000000 + Binary('edge[36]'))
((Binary('init_edge[1]') + Binary('term_edge[1]')) + Binary('edge[795]') + Binary('edge[743]') + Binary('edge[665]') + Binary('edge[663]') + Binary('edge[570]') + Binary('edge[520]') + Binary('edge[500]') + Binary('edge[470]') + Binary('edge[382]') + Binary('edge[348]') + Binary('edge[345]') + Binary('edge[308]') + Binary('edge[250]') + 0.000000 + Binary('edge[32]'))
((Binary('init_edge[2]') + Binary('term_edge[2]')) + Binary('edge[917]') + Binary('edge[710]') + Binary('edge[696]') + Binary('edge[645]') + Binary('edge[225]') + Binary('edge[212]') + Binary('edge[171]') + Binary('edge[127]') + Binary('edge[16]') + 0.000000 + Binary('edge[13]'))
((Binary('init_edge[3]') + Binary('term_edge[3]')) + Binary('edge[884]') + Binary('edge[778]') + Binary('edge[627]') + Binary('edge[3

In [810]:
total_exp = candy_exp + initial_node_exp + terminal_node_exp + all_node_deg_2_exp

exp = -total_exp

model = exp.compile()
bqm = model.to_bqm()

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.energy)

print(best_sample.array)
print(tail_graph.edges)

-180.0
<bound method PyCapsule.array of DecodedSolution({edge[0]:0, edge[484]:0, vertex[45]:1, edge[457]:0, edge[101]:0, init_edge[146]:0, edge[394]:0, vertex[44]:1, edge[456]:0, edge[100]:0, init_edge[147]:0, edge[395]:1, vertex[46]:1, edge[454]:0, edge[102]:1, init_edge[145]:0, edge[397]:0, vertex[47]:1, edge[455]:0, edge[103]:1, init_edge[144]:0, edge[396]:0, vertex[40]:1, edge[452]:0, edge[104]:0, init_edge[143]:0, edge[391]:0, vertex[41]:1, edge[453]:1, edge[105]:0, init_edge[142]:0, edge[390]:1, vertex[42]:1, edge[450]:0, edge[106]:0, init_edge[141]:0, edge[393]:0, vertex[43]:1, edge[451]:1, edge[107]:0, init_edge[140]:0, edge[392]:0, edge[108]:1, edge[109]:0, edge[131]:0, init_edge[110]:0, edge[265]:0, init_edge[91]:0, edge[10]:0, vertex[54]:1, edge[442]:0, term_edge[197]:0, edge[110]:0, init_edge[171]:0, vertex[55]:1, edge[443]:0, term_edge[196]:0, edge[111]:0, init_edge[170]:0, vertex[56]:1, edge[440]:0, term_edge[195]:0, edge[112]:0, init_edge[173]:0, vertex[57]:0, edge[441]: